In [73]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import pandas as pd
from tqdm import tqdm
import gensim
import pymorphy2
import functools as ft
from tqdm import tqdm
import string
import re

In [74]:
xy = pd.read_csv('X.csv')

In [75]:
Y = list(xy['Y'])

In [76]:
X = xy.drop('Y', axis=1)

In [77]:
def one_hot(arr):
    for i in range(len(arr)):
        langs = arr[i]
        oh_langs = [0 for i in range(13)]
        for lang in langs:
            oh_langs[lang] = 1
        arr[i] = oh_langs
    return arr

In [78]:
X['lang'] = list(map(lambda x: eval(x), list(X['lang'])))

In [79]:
X['lang'] = one_hot(list(X['lang']))

In [80]:
morph = pymorphy2.MorphAnalyzer()

In [81]:
def normalize_form(txt, morph):
    p = [morph.parse(x)[0].normal_form for x in txt]
    return p

In [82]:
def make_dict(x):
    dict_rus = ' '.join(x)
    translator = str.maketrans({key: ' ' for key in string.punctuation.replace('/', '').replace('-', '') + string.digits + '№'})
    dict_rus = dict_rus.translate(translator).lower()
    dict_rus_split = dict_rus.split()
    return dict_rus_split

In [83]:
def avg_feature_vector(words, model, num_features):
    featureVec = np.zeros((num_features,), dtype="float32")
    nwords = 0

    for word in words:
        nwords = nwords + 1
        try:
            featureVec = np.add(featureVec, model[word])
        except KeyError:
            featureVec = np.add(featureVec, np.zeros(num_features))

    if (nwords > 0):
        featureVec = np.divide(featureVec, nwords)
    return featureVec

In [84]:
X.fillna('', inplace=True)

In [85]:
w2v = gensim.models.KeyedVectors.load_word2vec_format(r'/home/anton/ank2prob/Предобученная моделька/ruscorpora.model.bin', binary = True)

In [86]:
def words2vecs(arr, w2v, morth):
    arr = list(map(lambda x: x.split(), arr))
    arr = [normalize_form(make_dict(new), morth) for new in arr]
    arr = [avg_feature_vector(sent, w2v, 300) for sent in list(arr)]
    return arr

In [87]:
X['projects'] = words2vecs(list(X['projects']), w2v, morph)

In [88]:
X['compet'] = words2vecs(list(X['compet']), w2v, morph)

In [89]:
projects = pd.DataFrame(list(X['projects']))

In [90]:
for i in range(len(list(X['projects'])[0])):
    X['p' + str(i)] = projects[i]

In [91]:
X = X.drop('projects', axis=1)

In [92]:
compets  = pd.DataFrame(list(X['compet']))

In [93]:
for i in range(len(list(X['compet'])[0])):
    X['c' + str(i)] = compets[i]

In [94]:
X = X.drop('compet', axis=1)

In [95]:
langs = pd.DataFrame(list(X['lang']))

In [96]:
for i in range(len(list(X['lang'])[0])):
    X['l' + str(i)] = langs[i]

In [97]:
X = X.drop('lang', axis=1)

In [98]:
X = X.drop('names', axis=1)

In [110]:
X['age'] = list(map(lambda x: 12 if x == '12.5' or x == 'Егор' else int(re.findall('\d+', x)[0]), list(X['age'])))

In [116]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

In [180]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [190]:
grc = GridSearchCV(LogisticRegression(), {'C':list(range(1, 10))})
grc.fit(X_train, Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [1, 2, 3, 4, 5, 6, 7, 8, 9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [191]:
grc.best_params_

{'C': 5}

In [192]:
lg = LogisticRegression(C=6, class_weight='balanced')
lg.fit(X_train, Y_train)

LogisticRegression(C=6, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [193]:
roc_auc_score(Y_test, list(map(lambda x: x[1], lg.predict_proba(X_test))))

0.76233333333333331